Мы владеем сетью магазинов, в которых продаются различные товары.  

Наши сотрудники собрали датасет из 8523 продаж товаров в различных магазинах нашей сети. Наша задача построить модель, предсказывающую продажи каждого продукта в конкретном магазине.  

Используя эту модель, попробовать выявить факторы, больше всего влияющие на увеличение продаж.  

Описание датасета:

Variable	Description  
- Item_Identifier	Id продукта  
- Item_Weight	Вес продукта  
- Item_Fat_Content	Содержание жира в продукте  
- Item_Visibility	%полок, отведенный под наш продукт в конкретном магазине  
- Item_Type	Категория продукта  
- Item_MRP	Максимальная цена продажи продукта  
- Outlet_Identifier	Идентификатор магазина  
- Outlet_Establishment_Year	Год открытия магазина  
- Outlet_Size	Площадь магазина  
- Outlet_Location_Type	Тип города, в котором расположен магазин  
- Outlet_Type	Признак является ли магазин продуктовым или супермаркетом  
- Item_Outlet_Sales	Продажи продукта в конкретном магазине. Именно ее и надо предсказывать  

В результате работы должен получиться:  
- Jupyter-ноутбук с моделью  
- Признаки, влияющие больше всего на уровень продаж  
- Датасет, если после ваших манипуляций он отличается от исходного;  
- Документ с обоснованием решения и краткими результатами: какие техники и почему использовали, что получили, что можно улучшить (можно в рамках jupyter notebook’а)

Баллы	Что надо сделать  
10	Провести EDA  
10	Обработать категориальные признаки   
10	Устранить пропущенные значения  
10	Изучить корреляцию признаков с данными о продажах  
10	Выбрать и обосновать метрику, на основе которой будем измерять качество полученной модели  
20	Построить и подобрать оптимальные параметры для любой линейной модели  
20	Построить и подобрать оптимальные параметры для любой нелинейной модели  
20	Провести стекинг нескольких моделей  
10	Оценить качество модели на отложенной выборке  
10	Выбрать топ 3 признака больше всего влияющие на объемы продаж  

Все шаги решения должны сопровождаться подробным описанием полученных результатов и обоснованием выбора того или иного шага!  
За отсутствие описания так же будут снижаться баллы.  

Максимальное доступное количество баллов - 130  
Для получения зачета надо набрать минимум 80 баллов  
Для получения зачета с отличием надо набрать минимум 120 баллов  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [4]:
data = pd.read_csv('data.csv')
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# Провести EDA

In [ ]:
# посмотрим глазами основные характеристики датасета

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [11]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Item_Weight,7060.0,12.857645,4.643456,4.555,8.773750,12.600000,16.850000,21.350000
Item_Visibility,8523.0,0.066132,0.051598,0.000,0.026989,0.053931,0.094585,0.328391
Item_MRP,8523.0,140.992782,62.275067,31.290,93.826500,143.012800,185.643700,266.888400
Outlet_Establishment_Year,8523.0,1997.831867,8.371760,1985.000,1987.000000,1999.000000,2004.000000,2009.000000
Item_Outlet_Sales,8523.0,2181.288914,1706.499616,33.290,834.247400,1794.331000,3101.296400,13086.964800


In [27]:
columns_cat = data.select_dtypes(object).columns
columns_digit = data.select_dtypes(exclude=object).columns

print(f'Количество категориальных переменных: {len(columns_cat)}')
print(f'Количество количественных переменных: {len(columns_digit)}')

Количество категориальных переменных: 7
Количество количественных переменных: 5


In [29]:
# много категорильных переменных, необходимо с ними поработать
# есть нулевые значения Item_Weight, Outlet_Size
# есть странный min=0 у признака Item_Visibility, а может и не странный

In [30]:
# Item_Visibility
data.Item_Visibility.value_counts()

0.000000    526
0.076975      3
0.041283      2
0.085622      2
0.187841      2
           ... 
0.092576      1
0.067544      1
0.115168      1
0.146896      1
0.050902      1
Name: Item_Visibility, Length: 7880, dtype: int64

In [6]:
data.Item_Identifier.value_counts()

FDG33    10
FDW13    10
DRN47     9
NCF42     9
FDF56     9
         ..
FDE52     1
FDK57     1
FDO33     1
FDC23     1
FDY43     1
Name: Item_Identifier, Length: 1559, dtype: int64

# Обработать категориальные признаки

# Устранить пропущенные значения

# Изучить корреляцию признаков с данными о продажах

# Выбрать и обосновать метрику, на основе которой будем измерять качество полученной модели

# Построить и подобрать оптимальные параметры для любой линейной модели

# Построить и подобрать оптимальные параметры для любой нелинейной модели

# Провести стекинг нескольких моделей

# Оценить качество модели на отложенной выборке

# Выбрать топ 3 признака больше всего влияющие на объемы продаж